In [1]:
#results might not be the same as on the paper. I somehown lost the trained model and parameters, I have used to generate original reuslts.

import json
import pandas as pd
import numpy as np
from sklearn.metrics import classification_report
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score

In [2]:
from google.colab import drive
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import hamming_loss


In [ ]:
#drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:



train_df = pd.read_csv('Data/train_2K.csv')

train_df['labels']= train_df[['bug_fix','feature_addition','performance_improvement']].values.tolist()

train_df=train_df[['text','labels']]

In [5]:
# temp=train_df[(train_df['bug_fix']==1) & (train_df['feature_addition']==1)].reset_index(drop=True)

KeyError: 'bug_fix'

                                                   text     labels
0         netfilter: xt_log: fix mark logging for ip...  [1, 0, 0]
1         [patch] inode-diet: eliminate i_blksize fr...  [1, 0, 0]
2         tensor roll op implementation (#14953)    ...  [1, 1, 0]
3                     improve video updates from sheet   [0, 0, 1]
4         [spark-9372] [sql] filter nulls in join ke...  [0, 0, 1]
...                                                 ...        ...
2032           preserve order of all heaps  fixes #795   [1, 0, 0]
2033      add base for app/polymer && add base for e...  [0, 1, 0]
2034      tty: ircomm  add tty_port          and use...  [0, 1, 0]
2035                         fix add_issues_modal_spec   [1, 0, 0]
2036      add 'renamefile' command to services (#235...  [0, 1, 0]

[2037 rows x 2 columns]


In [4]:
# temp.loc[82]['text']

NameError: name 'temp' is not defined

In [7]:
#train_df=train_df.drop(['id','bug_fix','feature_addition','performance_improvement'],axis=1)

#Test Train Split 
train, test = train_test_split(train_df, test_size=0.20)
#train, eval_df = train_test_split(train, test_size=0.15)


In [10]:
from simpletransformers.classification import MultiLabelClassificationModel

#,'pos_weight': [0.1,0.4,0.5]
# Create a MultiLabelClassificationModel

#distilbert-base-uncased
513,321,1185,
[0.25,0.15,0.59]
#'pos_weight': [2,3,0.59]
#train_batch_size
#'train_batch_size':  13,

#auto_weights

#
#model = MultiLabelClassificationModel('xlnet', 'xlnet-base-cased', num_labels=3, args={'train_batch_size':  8,'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 3,"save_model_every_epoch": True,"output_dir": "Data/outputs/"})
model = MultiLabelClassificationModel('distilbert', 'distilbert-base-uncased', num_labels=3, args={'train_batch_size':  8,'reprocess_input_data': True, 'overwrite_output_dir': True, 'num_train_epochs': 4,"save_model_every_epoch": True,"output_dir": "Data/outputs/"}, use_cuda=False)

#model = MultiLabelClassificationModel('reformer', 'reformer-enwik8', num_labels=3, args={ 'overwrite_output_dir': True, 'num_train_epochs': 3})
#ALBERT albert-base-v1
model.train_model(train)

Some weights of DistilBertForMultiLabelSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


  0%|          | 0/1629 [00:00<?, ?it/s]

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

Running Epoch 0 of 4:   0%|          | 0/204 [00:00<?, ?it/s]

Running Epoch 1 of 4:   0%|          | 0/204 [00:00<?, ?it/s]

Running Epoch 2 of 4:   0%|          | 0/204 [00:00<?, ?it/s]

Running Epoch 3 of 4:   0%|          | 0/204 [00:00<?, ?it/s]

(816, 0.26123214738133055)

In [11]:

#model_loaded = MultiLabelClassificationModel('distilbert', 'Data/multi_label/outputs/', args={})
result, model_outputs, wrong_predictions = model.eval_model(test)


  0%|          | 0/408 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/51 [00:00<?, ?it/s]

In [14]:
print(test["text"])

1623        lots of clean up and code review feedback ...
707         restore functionality to puppeteer url + f...
97          mm: simplify memory_isolation && compactio...
699         iwlwifi: mvm: add reorder timeout per fram...
1311        integrate healthcheck to all eshop service...
                              ...                        
830         staging: sep: add new pci identifier      ...
248         prefer `array?` rather than `array`       ...
204         - need to change a few things to exception...
1052        staging: move media drivers to staging/med...
173         [cs] fix conditional conformances in overl...
Name: text, Length: 408, dtype: object


In [15]:
#distill bert
#HL: 0.11

#bert
#HL:0.13

#prediction from our test dataset
predictions, raw_outputs = model.predict(test['text'].tolist())

print(classification_report(np.array(test['labels'].tolist()),np.array(predictions)))

print(hamming_loss(np.array(test['labels'].tolist()),np.array(predictions)))

  0%|          | 0/408 [00:00<?, ?it/s]

  0%|          | 0/51 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.88      0.92      0.90       158
           1       0.85      0.88      0.86       132
           2       0.85      0.80      0.83       179

   micro avg       0.86      0.86      0.86       469
   macro avg       0.86      0.87      0.86       469
weighted avg       0.86      0.86      0.86       469
 samples avg       0.88      0.89      0.87       469

0.10620915032679738


/Users/matthijs/Library/Caches/pypoetry/virtualenvs/pydrillertests-6uxJeVfr-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [ ]:
#HL: 0.22
#bert
#HL::0.26
weim_SAC_dataset=pd.read_excel('Data/NDSU/multi_label_commit_message/features_SAC2018.xlsx')

weim_SAC_dataset=weim_SAC_dataset[['Message','Corrective','Adaptive','Perfective']]
weim_SAC_dataset=weim_SAC_dataset.replace(to_replace=-1, value=0)
weim_SAC_dataset['labels']= weim_SAC_dataset[['Corrective','Adaptive','Perfective']].values.tolist()
weim_SAC_dataset=weim_SAC_dataset.drop(['Corrective','Adaptive','Perfective'],axis=1)

predictions_weim_SAC_dataset, raw_outputs_weim_dataset = model.predict(weim_SAC_dataset['Message'])

print(classification_report(np.array(weim_SAC_dataset['labels'].tolist()),np.array(predictions_weim_SAC_dataset)))
print("Hamming Loss")
print(hamming_loss(np.array(weim_SAC_dataset['labels'].tolist()),np.array(predictions_weim_SAC_dataset)))


              precision    recall  f1-score   support

           0       0.74      0.68      0.71       513
           1       0.78      0.29      0.43       621
           2       0.81      0.52      0.63      1185

   micro avg       0.78      0.50      0.61      2319
   macro avg       0.78      0.50      0.59      2319
weighted avg       0.79      0.50      0.59      2319
 samples avg       0.58      0.52      0.54      2319

Hamming Loss
0.2521974306964165


/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
#HL: 0.31
#bert
#HL:0.31
weim_msr_dataset=pd.read_csv('Data/msr_wiem.csv')
weim_msr_dataset=weim_msr_dataset[['text','corrective','adaptive','perfective']]
weim_msr_dataset['labels']= weim_msr_dataset[['corrective','adaptive','perfective']].values.tolist()
weim_msr_dataset=weim_msr_dataset.drop(['corrective','adaptive','perfective'],axis=1)
predictions_weim_msr_dataset, raw_outputs_weim_dataset = model.predict(weim_msr_dataset['text'].tolist())
print(classification_report(np.array(weim_msr_dataset['labels'].tolist()),np.array(predictions_weim_msr_dataset)))
print("Hamming Loss")
print(hamming_loss(np.array(weim_msr_dataset['labels'].tolist()),np.array(predictions_weim_msr_dataset)))


  0%|          | 0/967 [00:00<?, ?it/s]

  0%|          | 0/121 [00:00<?, ?it/s]

              precision    recall  f1-score   support

           0       0.67      0.56      0.61       394
           1       0.56      0.34      0.43       311
           2       0.39      0.66      0.49       259

   micro avg       0.52      0.52      0.52       964
   macro avg       0.54      0.52      0.51       964
weighted avg       0.56      0.52      0.52       964
 samples avg       0.50      0.50      0.49       964

Hamming Loss
0.31954498448810753


/Users/matthijs/Library/Caches/pypoetry/virtualenvs/pydrillertests-6uxJeVfr-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/matthijs/Library/Caches/pypoetry/virtualenvs/pydrillertests-6uxJeVfr-py3.10/lib/python3.10/site-packages/sklearn/metrics/_classification.py:1469: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
